In [1]:
!pip install transformers

import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
from transformers import BertTokenizer

from torchtext import data
from torch.utils.data import DataLoader, TensorDataset

from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

df=pd.read_csv('drive/MyDrive/nlp_proj/train_data.csv')
text=df['text']
label=df['label']

df2=pd.concat([label,text],axis=1)


Mounted at /content/drive


In [3]:
import re
emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                  "]+", re.UNICODE)
for i in range(len(df2)):
    df2['text'][i] = emoj.sub(r'', df2['text'][i])

<ipython-input-3-4f5eceffa938>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['text'][i] = emoj.sub(r'', df2['text'][i])


In [4]:
df2['text'] =  [re.sub(r'[?|$|.|!|।|,|:|;|]','', str(x)) for x in df2['text']]

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# print(stopwords.words('english'))
sw=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
stop_words = set(sw)
stop_word_parameter=list(sw)
stop_word_parameter_eng=list(stopwords.words('english'))
final_stop_words=stop_word_parameter+stop_word_parameter_eng
final_stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['अंदर',
 'अत',
 'अदि',
 'अप',
 'अपना',
 'अपनि',
 'अपनी',
 'अपने',
 'अभि',
 'अभी',
 'आदि',
 'आप',
 'इंहिं',
 'इंहें',
 'इंहों',
 'इतयादि',
 'इत्यादि',
 'इन',
 'इनका',
 'इन्हीं',
 'इन्हें',
 'इन्हों',
 'इस',
 'इसका',
 'इसकि',
 'इसकी',
 'इसके',
 'इसमें',
 'इसि',
 'इसी',
 'इसे',
 'उंहिं',
 'उंहें',
 'उंहों',
 'उन',
 'उनका',
 'उनकि',
 'उनकी',
 'उनके',
 'उनको',
 'उन्हीं',
 'उन्हें',
 'उन्हों',
 'उस',
 'उसके',
 'उसि',
 'उसी',
 'उसे',
 'एक',
 'एवं',
 'एस',
 'एसे',
 'ऐसे',
 'ओर',
 'और',
 'कइ',
 'कई',
 'कर',
 'करता',
 'करते',
 'करना',
 'करने',
 'करें',
 'कहते',
 'कहा',
 'का',
 'काफि',
 'काफ़ी',
 'कि',
 'किंहें',
 'किंहों',
 'कितना',
 'किन्हें',
 'किन्हों',
 'किया',
 'किर',
 'किस',
 'किसि',
 'किसी',
 'किसे',
 'की',
 'कुछ',
 'कुल',
 'के',
 'को',
 'कोइ',
 'कोई',
 'कोन',
 'कोनसा',
 'कौन',
 'कौनसा',
 'गया',
 'घर',
 'जब',
 'जहाँ',
 'जहां',
 'जा',
 'जिंहें',
 'जिंहों',
 'जितना',
 'जिधर',
 'जिन',
 'जिन्हें',
 'जिन्हों',
 'जिस',
 'जिसे',
 'जीधर',
 'जेसा',
 'जेसे',
 'जैसा',
 'जैसे',
 'जो',
 'तक',
 'तब',
 

In [6]:
df2['text'] = df2['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (final_stop_words)]))
df2.head()

,label,text
0,0,भीड़ हब्सी मिलेंगे
1,0,साले बेवकूफ मां मक्खियां हटा दें
2,0,बुर देदो मुह लंड ले लो
3,0,कुत्ता वहा चिल्ला तू क्यों
4,1,चाय पीता हूं मैं छोड़ ok


In [7]:
df4=df2.copy()
texts=df2['text'].to_list()

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
labels = {0:0,1:1}

In [8]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df4['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df4['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

np.random.seed(112)
df_train, df_val, df_test = np.split(df4.sample(frac=1, random_state=42),[int(.8*len(df)), int(.9*len(df))])


In [9]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [10]:
from torch.optim import Adam
from tqdm import tqdm
use_cuda = 1
def train(model, train_data, val_data, learning_rate, epochs):
    
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data)*10: .3f} % \
                | Val Accuracy: {total_acc_val / len(val_data)*10: .3f} %')

In [11]:
EPOCHS = 1

model = BertClassifier()
LR = 0.0001
              
train(model, df_train, df_val, LR, EPOCHS)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1262/1262 [28:12<00:00,  1.34s/it]


Epochs: 1 | Train Loss:  0.057                 | Train Accuracy:  0.648                 | Val Loss:  4.479 %                 | Val Accuracy:  51.462 %


In [12]:
# !pip install numba
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [12]:
test_data = pd.read_csv('drive/MyDrive/nlp_proj/hindi_test.csv')

for i in range(len(test_data)):
    test_data['text'][i] = emoj.sub(r'', test_data['text'][i])


test_data['text'] =  [re.sub(r'[?|$|.|!|।|,|:|;|]','', str(x)) for x in test_data['text']]
test_df = test_data

test_df['sentence_without_stopwords'] = test_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (final_stop_words)]))

In [ ]:
def intersection(df1,df2):
  intersect = np.intersect1d(df1,df2)
  print(len(intersect))

print(temp_train_x)
intersection(df_train,test_df['sentence_without_stopwords'])
intersection(df_val,test_df['sentence_without_stopwords'])

<h1> Testing </h1>

In [13]:
# -*- coding: utf-8 -*-


import warnings
warnings.filterwarnings('ignore')

df=pd.read_csv('drive/MyDrive/nlp_proj/hindi_test.csv')
text=df['text']
label=df['label']

df2=pd.concat([label,text],axis=1)

import re
emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                  "]+", re.UNICODE)
for i in range(len(df2)):
    df2['text'][i] = emoj.sub(r'', df2['text'][i])

df2['text'] =  [re.sub(r'[?|$|.|!|।|,|:|;|]','', str(x)) for x in df2['text']]

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# print(stopwords.words('english'))
sw=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
stop_words = set(sw)
stop_word_parameter=list(sw)
stop_word_parameter_eng=list(stopwords.words('english'))
final_stop_words=stop_word_parameter+stop_word_parameter_eng
final_stop_words

df2['text'] = df2['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (final_stop_words)]))
df2.head()

df4=df2.copy()
texts=df2['text'].to_list()

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
labels = {0:0,1:1}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df4['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df4['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

np.random.seed(112)
df_train, df_val, df_test = np.split(df4.sample(frac=1, random_state=42),[int(.8*len(df)), int(.9*len(df))])



from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

from torch.optim import Adam
from tqdm import tqdm
use_cuda = 1
def train(model, train_data, val_data, learning_rate, epochs):
    
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data)*10: .3f} % \
                | Val Accuracy: {total_acc_val / len(val_data)*10: .3f} %')



EPOCHS = 1

model = BertClassifier()
LR = 0.0001
              
train(model, df_train, df_val, LR, EPOCHS)

# !pip install numba
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

test_data = pd.read_csv('drive/MyDrive/nlp_proj/hindi_test.csv')

for i in range(len(test_data)):
    test_data['text'][i] = emoj.sub(r'', test_data['text'][i])


test_data['text'] =  [re.sub(r'[?|$|.|!|।|,|:|;|]','', str(x)) for x in test_data['text']]
test_df = test_data

test_df['sentence_without_stopwords'] = test_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (final_stop_words)]))

def intersection(df1,df2):
  intersect = np.intersect1d(df1,df2)
  print(len(intersect))

# print(temp_train_x)
intersection(df_train,test_df['sentence_without_stopwords'])
intersection(df_val,test_df['sentence_without_stopwords'])


Epochs: 1 | Train Loss:  0.068               | Train Accuracy:  0.610                | Val Loss:  5.672 %                 | Val Accuracy:  48.752 %
